In [1]:
!pip install pandas
!pip install ipython-sql prettytable 

import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [2]:
import pandas as pd
import sqlite3

# create / connect to SQLite database file
conn = sqlite3.connect("FinalDB.db")

In [7]:
%load_ext sql

In [9]:
census_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
chicago_public_schools = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
chicago_crime_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")

census_data.to_sql("census_data", conn, if_exists="replace", index=False )
chicago_public_schools.to_sql("chicago_public_schools", conn, if_exists="replace", index=False)
chicago_crime_data.to_sql("chicago_crime_data", conn, if_exists="replace", index=False)

533

In [10]:
%sql sqlite:///FinalDB.db

In [11]:
%sql select count("case_number") from chicago_crime_data

 * sqlite:///FinalDB.db
Done.


count(case_number)
533


In [20]:
%sql select "community_area_name", "community_area_number" from "census_data" where "per_capita_income" < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


In [82]:
%%sql

SELECT "Case_Number"
FROM "CHICAGO_CRIME_DATA"
WHERE "Primary_Type" LIKE '%MINOR%' 
   OR "Description" LIKE '%MINOR%' 
ORDER BY "Case_Number";

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HK238408
HL266884


In [89]:
%%sql select "case_number", "primary_type", description, "location_description" 
from chicago_crime_data 
where "Primary_type" like "%KIDNAP%" 
    and description like '%CHILD%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET


In [54]:
%sql select distinct("primary_type"), description, "location_description" from chicago_crime_data where "location_description" like '%SCHOOL%'

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION
BATTERY,SIMPLE,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,TO VEHICLE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,POSS: HEROIN(WHITE),"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,MANU/DEL:CANNABIS 10GM OR LESS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,TO LAND,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,BOMB THREAT,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,BOMB THREAT,"SCHOOL, PUBLIC, BUILDING"


In [38]:
%%sql 
select "Elementary, Middle, or High School" as School_type, avg(safety_score) as avg_score
        from chicago_public_schools
            group by school_type
            order by avg_score desc

 * sqlite:///FinalDB.db
Done.


School_type,avg_score
HS,49.62352941176471
ES,49.52038369304557
MS,48.0


In [56]:
%%sql

SELECT "Community_Area_number" AS Most_Crime_Prone_Area
FROM "CHICAGO_CRIME_DATA"
GROUP BY "Community_Area_number"
ORDER BY COUNT(*) DESC
LIMIT 1;


 * sqlite:///FinalDB.db
Done.


Most_Crime_Prone_Area
25.0


In [57]:
%%sql
select "community_area_name" from census_data 
order by percent_households_below_poverty desc limit 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale
Fuller Park
Englewood
North Lawndale
East Garfield Park


In [92]:
%%sql
select community_area_name, community_area_number from "census_data"
where hardship_index IN (
    select max("hardship_index") from "census_data")

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
Riverdale,54.0


In [91]:
%%sql

SELECT "COMMUNITY_AREA_NAME" AS Most_Crime_Prone_Area_Name
FROM "CENSUS_DATA"
WHERE "COMMUNITY_AREA_NUMBER" = (
    SELECT "Community_Area_number"
    FROM "CHICAGO_CRIME_DATA"
    GROUP BY "Community_Area_number"
    ORDER BY COUNT(*) DESC
    LIMIT 1
)

 * sqlite:///FinalDB.db
Done.


Most_Crime_Prone_Area_Name
Austin
